**This function is used to extract tbe label value of the string.**

As all the inputs are nested links in wiki data so its necessary to extaract the label using Web scraping.
Used BeautifulSoup

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pprint
import requests
# Defined a function to convert links to labels to put in text 
def ConverterFxn(input_Values):
  input_Values = input_Values.replace('entity','wiki')
  html = urlopen(input_Values)
  bsh = BeautifulSoup(html.read(), 'html.parser')
  x = bsh.h1
  x = str(x)
  # Converting link to usable form as extracted links are nested <Label type> and cant be used to web scrape
  Lidx = x.find('</span>')
  Iidx = x.find('wikibase-title-label') + 22
  New_string = ""
  for i in range(Iidx,Lidx):
    New_string = New_string + x[i]
  return(New_string)

In [ ]:
import pprint
from pydoc import describe
from re import S
import requests
# To Get Search Result we search the query o wiki data search box utilise the most relevant search
API_ENDPOINT = "https://www.wikidata.org/w/api.php"
query = input('Search Here ')

params = {
    'action': 'wbsearchentities',
    'format': 'json',
    'language': 'en',
    'search': query
}

r = requests.get(API_ENDPOINT, params = params)
Data = r.json()['search'][0]
print(r.json()['search'][0])

Search Here Narendra Modi
{'id': 'Q1058', 'title': 'Q1058', 'pageid': 1396, 'display': {'label': {'value': 'Narendra Modi', 'language': 'en'}, 'description': {'value': '15th Prime Minister of India', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q1058', 'concepturi': 'http://www.wikidata.org/entity/Q1058', 'label': 'Narendra Modi', 'description': '15th Prime Minister of India', 'match': {'type': 'label', 'language': 'en', 'text': 'Narendra Modi'}}


**Checking if the Data (initial) is working**

In [ ]:
# Checking the functionality of link of page from which we will extracct data
print(Data['concepturi'])

http://www.wikidata.org/entity/Q1058


Installing qwikidata api

In [ ]:
!pip3 install qwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for qwikidata: filename=qwikidata-0.4.2-py3-none-any.whl size=24886 sha256=c1ebe4de6fb088b6078049cda82063c4139d680d0d5e9e79a52776e6e526f42c
  Stored in directory: /root/.cache/pip/wheels/24/cc/fa/8bef9d5c734d2e64b900d40eea758d7c3dbf3b4ca6d0bb018f
Successfully built qwikidata


Installing mkwikidata

In [ ]:
!pip3 install mkwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


**Code to make/edit optimised queries for different links and instances**

In [ ]:

import pprint
import mkwikidata

import requests

query2 = '''
  { ?article  schema:about       ?item ;
              schema:inLanguage  "en" ;
              schema:isPartOf    <https://en.wikipedia.org/>
              '''              
query3 = f'''          
    FILTER ( ?item = <{Data['concepturi']}> )
    '''
query4 = ''' 
   OPTIONAL
      { ?item  wdt:P735  ?Name }
   OPTIONAL
      { ?item wdt:P21  ?Gender }
   OPTIONAL
      { ?item  wdt:P103  ?F_language }
   OPTIONAL
      { ?item  wdt:P569  ?Born_date }
   OPTIONAL
      { ?item  wdt:P570  ?RIP }
   OPTIONAL
      { ?item  wdt:P19  ?Birth_City }
   OPTIONAL
      { ?item  wdt:P27  ?Ntn }
   OPTIONAL
      { ?item  wdt:P22  ?Father_Name }
   OPTIONAL
      { ?item  wdt:P25  ?Mother_Name }
   OPTIONAL
      { ?item  wdt:P69  ?Person_Edu }
   OPTIONAL
      { ?item  wdt:P21  ?Pronoun }
   OPTIONAL
      { ?item  wdt:P106  ?Person_Work }
   OPTIONAL
      { ?item  wdt:P2218  ?NetWorth }
   OPTIONAL
      { ?item  wdt:P8687  ?SocialMedia_Followers }
   OPTIONAL
      { ?item  wdt:P140  ?Person_Religion }
    SERVICE wikibase:label
      { bd:serviceParam
                  wikibase:language  "en"
      }
  }
GROUP BY ?item ?itemLabel ?itemDescription
'''


url = 'https://query.wikidata.org/sparql'
query1 = f'''
PREFIX  schema: <http://schema.org/>
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT  ?item ?itemLabel ?itemDescription (SAMPLE(?Name) AS ?NameSample) (SAMPLE(?F_language) AS ?F_languageSample) (SAMPLE(?Person_Edu) AS ?Person_EduSample) (SAMPLE(?Pronoun) AS ?PronounSample) (SAMPLE(?Person_Work) AS ?Person_WorkSample) (SAMPLE(?Father_Name) AS ?Father_NameSample) (SAMPLE(?Mother_Name) AS ?Mother_NameSample) (SAMPLE(?Born_date) AS ?Born_dateSample) (SAMPLE(?Birth_City) AS ?Birth_CitySample) (SAMPLE(?article) AS ?articleSample)(SAMPLE(?RIP) AS ?RIPSample) (SAMPLE(?Ntn) AS ?NtnSample) (SAMPLE(?Person_Religion) AS ?Person_ReligionSample) (SAMPLE(?SocialMedia_Followers) AS ?SocialMedia_FollowersSample) (SAMPLE(?NetWorth) AS ?NetWorthSample) (SAMPLE(?Gender) AS ?GenderSample)
WHERE

'''
query = query1 + query2 + query3 + query4


r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()


**Checking if generated query is in perfect form**

In [ ]:
print(query)


PREFIX  schema: <http://schema.org/>
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT  ?item ?itemLabel ?itemDescription (SAMPLE(?Name) AS ?NameSample) (SAMPLE(?F_language) AS ?F_languageSample) (SAMPLE(?Person_Edu) AS ?Person_EduSample) (SAMPLE(?Pronoun) AS ?PronounSample) (SAMPLE(?Person_Work) AS ?Person_WorkSample) (SAMPLE(?Father_Name) AS ?Father_NameSample) (SAMPLE(?Mother_Name) AS ?Mother_NameSample) (SAMPLE(?Born_date) AS ?Born_dateSample) (SAMPLE(?Birth_City) AS ?Birth_CitySample) (SAMPLE(?article) AS ?articleSample)(SAMPLE(?RIP) AS ?RIPSample) (SAMPLE(?Ntn) AS ?NtnSample) (SAMPLE(?Person_Religion) AS ?Person_ReligionSample) (SAMPLE(?SocialMedia_Followers) AS ?SocialMedia_FollowersSample) (SAMPLE(?NetWorth) AS ?NetWorthSample) (SAMPLE(?Gender) AS ?GenderSample)
WHERE


  { ?article  schema:about       ?item ;
              schema:inLanguage  "en" ;
              sche

**Checking extracted data extracted with help of the query**

In [ ]:
pprint.pprint(data)

{'head': {'vars': ['item',
                   'itemLabel',
                   'itemDescription',
                   'NameSample',
                   'F_languageSample',
                   'Person_EduSample',
                   'PronounSample',
                   'Person_WorkSample',
                   'Father_NameSample',
                   'Mother_NameSample',
                   'Born_dateSample',
                   'Birth_CitySample',
                   'articleSample',
                   'RIPSample',
                   'NtnSample',
                   'Person_ReligionSample',
                   'SocialMedia_FollowersSample',
                   'NetWorthSample',
                   'GenderSample']},
 'results': {'bindings': [{'Birth_CitySample': {'type': 'uri',
                                                'value': 'http://www.wikidata.org/entity/Q2570726'},
                           'Born_dateSample': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                       

**Checking nested dictionary's functionality**

In [ ]:
print(data['head']['vars'][8])

Father_NameSample


In [ ]:
# print(data['results']['bindings'][0]['NameSample']['value'])

**Converting data to usable form: **

1.   Convertion of extracted data to usable variables.
2.   Converting the data links to labels that could be used in text
1.   Making sentences for the newly made variables.
2.   printing of the variables 

Note : If any of the existing steps fails the variable will be skipped hence either the data is not present or non-editable.

*All the variables that were required in all the groups are pre-labeled.*

In [ ]:

Name = data['results']['bindings'][0]['itemLabel']['value']
Gender = data['results']['bindings'][0]['GenderSample']['value']
Gender = ConverterFxn(Gender)
if(Gender == 'male'):
  Pronoun = 'He'
  Prenoun = 'his' 
if(Gender == 'female'):
  Pronoun = 'She'
  Prenoun = 'her'
if(Gender != 'female' and Gender != 'male'):
  Pronoun = 'They'
  Prenoun = 'them'

try:
  Person_Description = data['results']['bindings'][0]['itemDescription']['value']
  Description = f'''{Name} is {Person_Description}.'''
  print(Description,end = ' ')
except:
  pass

try:
  Born_date = data['results']['bindings'][0]['Born_dateSample']['value']
  Person_DOB = f'''{Pronoun} was born on {Born_date}.'''
  print(Person_DOB,end = ' ')

except:
  pass  

try:
  Birth_City = data['results']['bindings'][0]['Birth_CitySample']['value']
  Birth_City = ConverterFxn(Birth_City)
  Place_Of_Birth = f'''{Pronoun} was born in city of {Birth_City}.'''
  print(Place_Of_Birth,end = ' ')
except:
  pass  

try:
  Person_Education = data['results']['bindings'][0]['Person_EduSample']['value']
  Person_Education = ConverterFxn(Person_Education)
  Person_CATEGORY_Education = f'''{Name} got {Prenoun} formal education for {Person_Education}.'''
  print(Person_CATEGORY_Education,end = ' ')
except:
  pass

try:
  F_language = data['results']['bindings'][0]['F_languageSample']['value']
  F_language = ConverterFxn(F_language)
  First_CATEGORY_language = f'''{Pronoun} speaks {F_language} in day to day life.'''
  print(First_CATEGORY_language,end = ' ')
except:
  pass  

try:
  Father_Name = data['results']['bindings'][0]['Father_NameSample']['value']
  Father_Name = ConverterFxn(Father_Name)
  Mother_Name = data['results']['bindings'][0]['Mother_NameSample']['value']
  Mother_Name = ConverterFxn(Mother_Name)
  Family_Parents = f'''{Prenoun} family include father {Father_Name} and Mother {Mother_Name}.'''
  print(Family_Parents,end = ' ')
except:
  pass  
try:
  Old_Name= data['results']['bindings'][0]['NameSample']['value']
  Old_Name = ConverterFxn(Old_Name)
  Common_Name = f'''{Pronoun} is also well know by name {Old_Name}.'''
  print(Common_Name,end = ' ')
except:
  pass
try:
  Ntn = data['results']['bindings'][0]['NtnSample']['value']
  Ntn = ConverterFxn(Ntn)
  Person_Citizenship = f'''{Pronoun} is a proud citizen of {Ntn}.'''
  print(Person_Citizenship,end = ' ')
except:
  pass
try:
  Person_Religion = data['results']['bindings'][0]['Person_ReligionSample']['value']
  Person_Religion = ConverterFxn(Person_Religion)
  Person_CATEGORY_Religion = f'''{Pronoun} follows {Person_Religion}.'''
  print(Person_CATEGORY_Religion,end = ' ')
except:
  pass
try:
  Person_Work = data['results']['bindings'][0]['Person_WorkSample']['value']
  Person_Work = ConverterFxn(Person_Work)
  Occupation = f'''{Prenoun} occupation as {Person_Work}.'''
  print(Occupation,end = ' ')
except:
  pass


try:
  NetWorth = data['results']['bindings'][0]['NetWorthSample']['value']
  Person_NetWorth = f'''{Name} is proud owner of NetWorth equating {NetWorth}.'''
  print(Person_NetWorth,end = ' ')
except:
  pass
try:
  SocialMedia_Followers = NetWorth = data['results']['bindings'][0]['SocialMedia_FollowersSample']['value']
  Person_SocialMedia_CATEGORY_Followers = f'''{Pronoun} has a individual social media followe count of {SocialMedia_Followers}.'''
  print(Person_SocialMedia_CATEGORY_Followers,end = ' ')
except:
  pass

try:
  RIP = data['results']['bindings'][0]['RIPSample']['value']
  END_Game = f'''{Pronoun} passed away on {RIP}'''
  print(END_Game,end = ' ')
except:
  pass

Narendra Modi is 15th Prime Minister of India. He was born on 1950-09-17T00:00:00Z. He was born in city of Vadnagar. Narendra Modi got his formal education for Gujarat University. He speaks Gujarati in day to day life. his family include father Damodardas Mulchand Modi and Mother Heeraben Modi. He is also well know by name Damodar. He is a proud citizen of India. He follows Sufism. his occupation as writer. Narendra Modi is proud owner of NetWorth equating 391937. He has a individual social media followe count of 10500000. 

*Note : This code gives a one time output.*